In [4]:
!pip install selenium

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import csv
import os
import time

In [6]:
service = Service(executable_path="./chromedriver")
####

######
driver = webdriver.Chrome(service=service)
csv_file_path = "job_data_Bechtel.csv"

# Check if the file exists and is empty
file_exists = os.path.isfile(csv_file_path)
file_empty = os.stat(csv_file_path).st_size == 0 if file_exists else True

# Write the header row if the file doesn't exist or is empty
if not file_exists or file_empty:
    with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Job Title", "Job Description"])
#https://jobs.bechtel.com/go/Energy-Jobs-in-Washington-DC/357495/
driver.get("https://jobs.bechtel.com/go/Energy-Jobs-in-Washington-DC/357495/")





# Function to get elements with retry
def get_elements_with_retry(driver, by, value, max_attempts=3):
    attempts = 0
    while attempts < max_attempts:
        try:
            elements = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((by, value))
            )
            return elements
        except StaleElementReferenceException:
            attempts += 1
    return []

# Function to get element text with retry
def get_element_text_with_retry(element, max_attempts=3):
    attempts = 0
    while attempts < max_attempts:
        try:
            return element.text
        except StaleElementReferenceException:
            attempts += 1
    return ""

job_id = "jobTitle-link"
jobs = get_elements_with_retry(driver, By.CLASS_NAME, job_id)

#job
for j in jobs:
    try:
        job_title = get_element_text_with_retry(j)
        j.click()
        
        # Wait for job description to load
        job_description_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "job"))
        )
        job_description = get_element_text_with_retry(job_description_element)
        
        with open(csv_file_path, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow([job_title, job_description])
        
        # Navigate back to the job list
        driver.back()
        
        # Wait for the job list to reload
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, job_id))
        )
        
        # Re-fetch the job elements
        jobs = get_elements_with_retry(driver, By.CLASS_NAME, job_id)
    except Exception as e:
        print(f"Error processing job: {e}")
        continue

driver.quit()


Error processing job: Message: stale element reference: stale element not found
  (Session info: chrome-headless-shell=127.0.6533.89); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000102656a80 chromedriver + 4385408
1   chromedriver                        0x000000010264f38c chromedriver + 4354956
2   chromedriver                        0x000000010226cb0c chromedriver + 281356
3   chromedriver                        0x000000010227a898 chromedriver + 338072
4   chromedriver                        0x0000000102271d84 chromedriver + 302468
5   chromedriver                        0x0000000102270678 chromedriver + 296568
6   chromedriver                        0x0000000102272e58 chromedriver + 306776
7   chromedriver                        0x0000000102272ed0 chromedriver + 306896
8   chromedriver                        0x00000001